In [ ]:
import sqlite3
import requests

In [ ]:
conn = sqlite3.connect('DataBase.db')
print("Подключение к базе данных произошло успешно")
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        title TEXT,
        body TEXT
    )
''')

conn.commit()
print("Таблица posts создана.")

In [ ]:
response = requests.get('https://jsonplaceholder.typicode.com/posts')

if response.status_code == 200:
    posts = response.json()
    print("Данные получены.")
else:
    print(f"Ошибка запроса: {response.status_code}")

In [ ]:
def clear_posts_table():
    cursor.execute('DELETE FROM posts')
    conn.commit()
    print("Таблица posts очищена.")

clear_posts_table()

In [ ]:
for post in posts:
    cursor.execute('''
        SELECT COUNT(*) FROM posts WHERE id = ?
    ''', (post['id'],))

    count = cursor.fetchone()[0]

    if count == 0:
        cursor.execute('''
            INSERT INTO posts (id, user_id, title, body)
            VALUES (?, ?, ?, ?)
        ''', (post['id'], post['userId'], post['title'], post['body']))
        print(f"Добавлен пост с id {post['id']}.")
    else:
        print(f"Пост с id {post['id']} уже существует в базе данных.")

conn.commit()
print("Данные сохранены в базу данных.")

In [ ]:
def get_posts_by_user(user_id):
    cursor.execute('''
        SELECT * FROM posts WHERE user_id = ?
    ''', (user_id,))
    return cursor.fetchall()

user_id = 1
user_posts = get_posts_by_user(user_id)

if user_posts:
    for post in user_posts:
        print(f"Post ID: {post[0]}, Title: {post[2]}, Body: {post[3]}")
else:
    print(f"Посты для user_id {user_id} не найдены.")

conn.close()